# Fine-tune Whisper Small on TORGO with LoRA

This notebook fine-tunes **Whisper Small** on the preprocessed TORGO dataset using **LoRA** (Low-Rank Adaptation) for dysarthric speech recognition.

**Supports both Google Colab (GPU) and local (MPS/CPU).**

### Colab Setup
1. Upload your `audio/torgo/processed/` folder to Google Drive under `My Drive/ASR Project/`
   - The folder should contain `train/`, `validation/`, `test/` subfolders with `.wav` files
   - Also include `metadata.json` inside `processed/`
2. Select a **T4 GPU** runtime: Runtime → Change runtime type → T4 GPU
3. Run all cells

### Local Setup
1. Run `audio/data_loader.ipynb` to download TORGO
2. Run `audio/torgo_preprocessing.ipynb` to generate processed audio + metadata
3. Run this notebook from the **project root**

In [ ]:
import os, sys

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    print("Running on Google Colab")
    !pip install -q transformers datasets evaluate peft accelerate jiwer soundfile librosa
    from google.colab import drive
    drive.mount("/content/drive")
    PROJECT_ROOT = "/content"
    DATA_ROOT = "/content/drive/MyDrive/ASR Project"
    os.makedirs(PROJECT_ROOT, exist_ok=True)
else:
    print("Running locally")
    from pathlib import Path
    PROJECT_ROOT = str(Path(os.getcwd()).resolve())
    while PROJECT_ROOT != "/" and not os.path.exists(os.path.join(PROJECT_ROOT, "asr", "config.yaml")):
        PROJECT_ROOT = str(Path(PROJECT_ROOT).parent)
    os.chdir(PROJECT_ROOT)
    DATA_ROOT = PROJECT_ROOT

print(f"Project root: {PROJECT_ROOT}")
print(f"Data root:    {DATA_ROOT}")

In [ ]:
import torch

if IN_COLAB:
    device = "cuda" if torch.cuda.is_available() else "cpu"
else:
    device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"

config = {
    "model": {
        "name": "openai/whisper-small",
        "language": "en",
        "task": "transcribe",
    },
    "lora": {
        "r": 16,
        "alpha": 32,
        "dropout": 0.1,
        "target_modules": ["q_proj", "v_proj", "k_proj", "out_proj"],
    },
    "data": {
        "processed_dir": os.path.join(DATA_ROOT, "audio", "torgo", "processed"),
        "metadata_path": os.path.join(DATA_ROOT, "audio", "torgo", "processed", "metadata.json"),
        "sampling_rate": 16000,
        "max_audio_length": 15.0,
        "include_augmented": True,
        "dysarthric_only": False,
    },
    "output": {
        "model_dir": os.path.join(PROJECT_ROOT, "asr_checkpoints", "whisper-lora"),
        "log_dir": os.path.join(PROJECT_ROOT, "asr_runs"),
    },
}

if device == "cuda":
    train_cfg = {
        "epochs": 2,
        "batch_size": 8,
        "gradient_accumulation_steps": 2,
        "learning_rate": 3e-4,
        "warmup_steps": 50,
        "fp16": True,
        "eval_steps": 500,
        "save_steps": 500,
        "logging_steps": 25,
        "dataloader_num_workers": 2,
    }
elif device == "mps":
    train_cfg = {
        "epochs": 2,
        "batch_size": 1,
        "gradient_accumulation_steps": 16,
        "learning_rate": 3e-4,
        "warmup_steps": 50,
        "fp16": False,
        "eval_steps": 250,
        "save_steps": 250,
        "logging_steps": 25,
        "dataloader_num_workers": 0,
    }
else:
    train_cfg = {
        "epochs": 2,
        "batch_size": 2,
        "gradient_accumulation_steps": 8,
        "learning_rate": 3e-4,
        "warmup_steps": 50,
        "fp16": False,
        "eval_steps": 250,
        "save_steps": 250,
        "logging_steps": 25,
        "dataloader_num_workers": 0,
    }

effective_batch = train_cfg["batch_size"] * train_cfg["gradient_accumulation_steps"]
print(f"Device: {device}")
print(f"Batch: {train_cfg['batch_size']} x {train_cfg['gradient_accumulation_steps']} = {effective_batch} effective")
print(f"FP16: {train_cfg['fp16']}")

## Load Preprocessed Data

Reads the metadata JSON and constructs a HuggingFace Dataset from the processed `.wav` files. Audio is loaded lazily (only decoded when accessed), so this step is fast and lightweight.

In [ ]:
import json, re
from datasets import Dataset, DatasetDict, Audio

data_cfg = config["data"]
processed_dir = data_cfg["processed_dir"]
metadata_path = data_cfg["metadata_path"]

assert os.path.exists(metadata_path), f"Metadata not found at {metadata_path}"

with open(metadata_path) as f:
    metadata = json.load(f)

include_augmented = data_cfg["include_augmented"]
dysarthric_only = data_cfg["dysarthric_only"]
AUGMENT_PATTERN = re.compile(r"sample_\d{5}_.+\.wav")

def load_split(split_name):
    split_meta = metadata[split_name]
    audio_paths, transcriptions = [], []
    for filename, meta in split_meta.items():
        if dysarthric_only and meta["speech_status"] != "dysarthria":
            continue
        if not include_augmented and AUGMENT_PATTERN.match(filename):
            continue
        wav_path = os.path.join(processed_dir, split_name, filename)
        if os.path.exists(wav_path) and meta["transcription"]:
            audio_paths.append(wav_path)
            transcriptions.append(meta["transcription"])
    ds = Dataset.from_dict({"audio": audio_paths, "transcription": transcriptions})
    ds = ds.cast_column("audio", Audio(sampling_rate=data_cfg["sampling_rate"]))
    return ds

dataset = DatasetDict({
    "train": load_split("train"),
    "validation": load_split("validation"),
    "test": load_split("test"),
})

print(f"Augmented: {'included' if include_augmented else 'excluded'}")
print(f"Filter:   {'dysarthric only' if dysarthric_only else 'all speakers'}")
print(f"Train: {len(dataset['train']):,}  Val: {len(dataset['validation']):,}  Test: {len(dataset['test']):,}")

In [ ]:
from transformers import AutoProcessor

model_name = config["model"]["name"]
processor = AutoProcessor.from_pretrained(
    model_name,
    language=config["model"]["language"],
    task=config["model"]["task"],
)
print("Processor ready.")

## On-the-fly Feature Extraction

Instead of precomputing Whisper mel spectrograms for all ~92K samples (which requires ~88 GB of storage and RAM), the data collator extracts features **on-the-fly** during training. Only one batch of features is ever in memory at a time.

In [ ]:
import torch
from dataclasses import dataclass
from typing import Any

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: list[dict]) -> dict[str, torch.Tensor]:
        input_features = []
        for f in features:
            audio = f["audio"]
            mel = self.processor.feature_extractor(
                audio["array"], sampling_rate=audio["sampling_rate"]
            ).input_features[0]
            input_features.append({"input_features": mel})

        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [
            {"input_ids": self.processor.tokenizer(f["transcription"]).input_ids}
            for f in features
        ]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        labels = labels_batch["input_ids"].masked_fill(
            labels_batch.attention_mask.ne(1), -100
        )
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)
print("Data collator ready (on-the-fly feature extraction).")

## Model Setup

Load Whisper Small and apply LoRA adapters. Gradient checkpointing trades compute for memory — essential for fitting on consumer GPUs.

In [ ]:
from transformers import WhisperForConditionalGeneration
from peft import LoraConfig, get_peft_model

model = WhisperForConditionalGeneration.from_pretrained(model_name)
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

lora_cfg = config["lora"]
lora_config = LoraConfig(
    r=lora_cfg["r"],
    lora_alpha=lora_cfg["alpha"],
    target_modules=lora_cfg["target_modules"],
    lora_dropout=lora_cfg["dropout"],
    bias="none",
)
model = get_peft_model(model, lora_config)
model.gradient_checkpointing_enable()
model.print_trainable_parameters()
print(f"Device: {device}")

In [ ]:
import numpy as np
import evaluate

wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids = np.where(label_ids != -100, label_ids, processor.tokenizer.pad_token_id)
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

## Training

The trainer handles device placement, gradient accumulation, mixed precision, evaluation, and checkpointing automatically.

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, TrainerCallback, TrainerState, TrainerControl, TrainingArguments, EarlyStoppingCallback
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR

class SavePeftModelCallback(TrainerCallback):
    def on_save(self, args: TrainingArguments, state: TrainerState, control: TrainerControl, **kwargs):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")
        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)
        pytorch_bin = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_bin):
            os.remove(pytorch_bin)
        return control

output_cfg = config["output"]
os.makedirs(output_cfg["model_dir"], exist_ok=True)
os.makedirs(output_cfg["log_dir"], exist_ok=True)

steps_per_epoch = len(dataset["train"]) // effective_batch
total_steps = steps_per_epoch * train_cfg["epochs"]
print(f"Steps/epoch: ~{steps_per_epoch:,}  |  Total: ~{total_steps:,}")

training_args = Seq2SeqTrainingArguments(
    output_dir=output_cfg["model_dir"],
    logging_dir=output_cfg["log_dir"],
    num_train_epochs=train_cfg["epochs"],
    per_device_train_batch_size=train_cfg["batch_size"],
    per_device_eval_batch_size=train_cfg["batch_size"],
    learning_rate=train_cfg["learning_rate"],
    warmup_steps=train_cfg["warmup_steps"],
    fp16=train_cfg["fp16"],
    eval_strategy="steps",
    eval_steps=train_cfg["eval_steps"],
    save_steps=train_cfg["save_steps"],
    gradient_accumulation_steps=train_cfg["gradient_accumulation_steps"],
    load_best_model_at_end=True,
    metric_for_best_model="eval_wer",
    greater_is_better=False,
    generation_max_length=225,
    logging_steps=train_cfg["logging_steps"],
    remove_unused_columns=False,
    label_names=["labels"],
    save_total_limit=3,
    dataloader_pin_memory=(device == "cuda"),
    dataloader_num_workers=train_cfg["dataloader_num_workers"],
    report_to="none",
)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
    processing_class=processor.feature_extractor,
    compute_metrics=compute_metrics,
    callbacks=[
        SavePeftModelCallback(),
        EarlyStoppingCallback(early_stopping_patience=3),
    ],
)
model.config.use_cache = False
print("Trainer ready.")

In [ ]:
trainer.train()

In [ ]:
final_dir = os.path.join(output_cfg["model_dir"], "final_adapter")
os.makedirs(final_dir, exist_ok=True)
trainer.save_model(final_dir)
print(f"Saved final LoRA adapter to {final_dir}")

if IN_COLAB:
    drive_save = os.path.join(DATA_ROOT, "asr_checkpoints", "whisper-lora-final")
    os.makedirs(drive_save, exist_ok=True)
    import shutil
    shutil.copytree(final_dir, drive_save, dirs_exist_ok=True)
    print(f"Copied to Google Drive: {drive_save}")